## 各种import

In [1]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("..\examples"))

import utils; reload(utils)
from utils import *

from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop

Using Theano backend.


## 从Keras导入数据

In [2]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000L, 28L, 28L), (60000L,), (10000L, 28L, 28L), (10000L,))

## 处理数据

### One-Hot-Encoding

In [3]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道

In [4]:
trn_data = np.expand_dims(trn_data, 1)
test_data = np.expand_dims(test_data, 1)
trn_data.shape

(60000L, 1L, 28L, 28L)

### 对输入图像进行正则化

In [5]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

## 1. 简单神经网络模型

In [10]:
simpleNN = Sequential()
simpleNN.add(Lambda(norm_input, input_shape=(1, 28, 28), output_shape=(1, 28, 28)))
simpleNN.add(Flatten())
simpleNN.add(Dense(392, activation='softmax'))
simpleNN.add(Dense(10, activation='softmax'))
simpleNN.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

### 1.1 直接训练

In [9]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 34s - loss: 1.4415 - acc: 0.4401 - val_loss: 1.2779 - val_acc: 0.4703


In [ ]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 103s - loss: 0.8987 - acc: 0.6784 - val_loss: 0.7461 - val_acc: 0.7462
Epoch 2/3
60000/60000 [==============================] - 65s - loss: 0.7182 - acc: 0.7699 - val_loss: 0.6953 - val_acc: 0.7922
Epoch 3/3
 8416/60000 [===>..........................] - ETA: 46s - loss: 0.6647 - acc: 0.8019

### 1.2 生成batch再训练

利用ImageDataGenerator(), 使用fit_generator而不是fit
注意fit_generator的文档中参数由于版本升级发生了变化

In [32]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

In [52]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 18s - loss: 0.2832 - acc: 0.9273 - val_loss: 0.3078 - val_acc: 0.9265


In [54]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/3
60000/60000 [==============================] - 20s - loss: 0.2186 - acc: 0.9448 - val_loss: 0.2648 - val_acc: 0.9351
Epoch 2/3
60000/60000 [==============================] - 23s - loss: 0.2065 - acc: 0.9476 - val_loss: 0.2312 - val_acc: 0.9435
Epoch 3/3
60000/60000 [==============================] - 23s - loss: 0.1947 - acc: 0.9511 - val_loss: 0.2390 - val_acc: 0.9396


## 2. 类似vgg16的卷积神经网络模型

In [83]:
simpleCNN = Sequential()
simpleCNN.add(Lambda(norm_input, input_shape=(1, 28, 28), output_shape=(1, 28, 28)))
simpleCNN.add(ZeroPadding2D())
simpleCNN.add(Convolution2D(64, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(ZeroPadding2D())
simpleCNN.add(Convolution2D(128, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(ZeroPadding2D())
simpleCNN.add(Convolution2D(256, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(Flatten())
simpleCNN.add(Dense(512, activation="softmax"))
simpleCNN.add(Dense(10, activation="softmax"))
simpleCNN.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

C:\anaconda2\lib\site-packages\keras\layers\core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_19 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [86]:
simpleCNN = Sequential()
simpleCNN.add(Lambda(norm_input, input_shape=(1, 28, 28), output_shape=(1, 28, 28)))
simpleCNN.add(Convolution2D(32, 3, 3, activation="relu"))
simpleCNN.add(Convolution2D(32, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())
simpleCNN.add(Convolution2D(64, 3, 3, activation="relu"))
simpleCNN.add(Convolution2D(64, 3, 3, activation="relu"))
simpleCNN.add(MaxPooling2D())

simpleCNN.add(Flatten())
simpleCNN.add(Dense(512, activation="relu"))
simpleCNN.add(Dense(10, activation="softmax"))
simpleCNN.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [88]:
simpleCNN.optimizer.lr = 0.1
simpleCNN.fit_generator(batches, 6000, nb_epoch=1, validation_data=test_batches, nb_val_samples=300)

Epoch 1/1
6016/6000 [==============================] - 17s - loss: 14.4222 - acc: 0.1052 - val_loss: 14.4559 - val_acc: 0.1031


In [81]:
simpleCNN.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_18 (Lambda)               (None, 1, 28, 28)     0           lambda_input_17[0][0]            
____________________________________________________________________________________________________
zeropadding2d_24 (ZeroPadding2D) (None, 1, 30, 30)     0           lambda_18[0][0]                  
____________________________________________________________________________________________________
convolution2d_23 (Convolution2D) (None, 32, 28, 28)    320         zeropadding2d_24[0][0]           
____________________________________________________________________________________________________
maxpooling2d_21 (MaxPooling2D)   (None, 32, 14, 14)    0           convolution2d_23[0][0]           
___________________________________________________________________________________________